In [3]:
import os
import sys
from math import pi, sqrt
import numpy as np
from numpy.random import rand
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad
from torch.autograd import Variable
from random import random, normalvariate
from sklearn.neighbors import NearestNeighbors

from scipy.interpolate import RegularGridInterpolator

from model.utility import generate_data,generate_data_Lag, histogramcnn, histtopdf,make_grid, cal_grid_data
from model.LLES import kernel, LLES

from copy import copy
torch.set_default_dtype(torch.float64)

# Train the smoothing kernel first before training L-LES  model

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Lagrangian data

Load the trajectory of the Lagrangian particle to the variable pos_traj(position), vel_traj(velocity), rho_traj(density)

They should have the shape of (ntime, nparticle, 3) for velocity and position or (ntime, nparticle, 1) for density. 

In this notebook, we use ntime =2, nparticle = 262144

In [13]:
def read_HIT_data(device):
    loadData = np.load('dataset/HIT_Mt008_train.npz', allow_pickle=True)
    pos_traj = loadData['pos']
    vel_traj = loadData['vel']
    rho_traj = loadData['rho']
    return torch.from_numpy(pos_traj).to(device), torch.from_numpy(vel_traj).to(device), torch.from_numpy(rho_traj).to(device)

In [14]:
# Read HIT data for Mt=0.08
pos_traj, vel_traj, rho_traj = read_HIT_data(device)

In [15]:
#Obtain the reference scale oof the velocity and density
vref = (vel_traj.var())**0.5
rhoref = (rho_traj.var())**0.5

In [16]:
N = 262144  ## 64**3
dt = 0.05
tref = torch.tensor(0.1) # Should the be reference timescale at the filtered scale
nfield = 32
nfieldt = nfield**3
neighbor_kernel = 60


In [17]:
#Load the pre-trained kernel function
kernel_wnn = kernel(N, nfield, neighbor_kernel, device)
kernel_wnn.load_state_dict(torch.load("SmoothingKernel.params"))

<All keys matched successfully>

In [18]:
model  = LLES(device, N,dt, vref, tref,rhoref, nfieldt, kernel_wnn ,neighbor_kernel)

In [19]:
traj_train_label_vel = ((vel_traj[1:]-vel_traj[:-1])/dt)
traj_train_label_rho = ((rho_traj[1:]-rho_traj[:-1])/dt)

# Setup Statistics-based Loss function

Build kerenl function that maps samples to histogram.

In [20]:
#Velocity histogram
accl_max = ((vel_traj[1:]-vel_traj[:-1])/dt).abs().max()
datatohist = histogramcnn(accl_max)
datatohist.to(device)
datatohist.set_param(device)

#Density histogram
rho_max = ((rho_traj[1:]-rho_traj[:-1])/dt).abs().max()
datatohist_rho = histogramcnn(rho_max)
datatohist_rho.to(device)
datatohist_rho.set_param(device)

# Setup Eulerian-grid based Loss function

In [21]:
# Generate field data for training
grid = make_grid(nfield)
grid_field_ref, w_field, p_rho, fneighbor=cal_grid_data(pos_traj[0],vel_traj[0],rho_traj[0],grid,kernel_wnn,neighbor_kernel)
field_train_label = torch.tensor(grid_field_ref).to(device)
w_field = torch.from_numpy(w_field).to(device)

Generating GT field data


In [22]:
# Compute the neighbors list of particles and grid points
out = model.update_neighborlist_sklearn(pos_traj[0],fneighbor)

In [23]:
# Configure the coeffcients of different loss function and optimizer
alpha_field = 1.0
alpha_kl = 0.1
alpha_traj = 1.0
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)

In [24]:
# Setup minibatch for field-based and trajectory-based loss function
batch_size_f =32*8
batch_number_f = int(nfieldt/batch_size_f)
batch_array_f = np.arange(nfieldt)
np.random.shuffle(batch_array_f)
batch_array_f=torch.tensor(batch_array_f.reshape([batch_number_f,batch_size_f]))
#-------traj batch ------------                                                
batch_size_t =128*32
batch_number_t = int(N/batch_size_t)
batch_array_t = np.arange(N)
np.random.shuffle(batch_array_t)
batch_array_t=torch.tensor(batch_array_t.reshape([batch_number_t,batch_size_t]))


# First train with Trajectory-based and Statistics-based loss

In [25]:
train_epochs = 1
loss_traj = []

In [26]:
for i in range(train_epochs):
    for nb in range(batch_number_t):
        optimizer.zero_grad()
        accl = model.cal_a_nn(pos_traj[0],vel_traj[0],rho_traj[0],batch_array_t[nb])
    
        # Trajectory-based loss function
        accl_gt = traj_train_label_vel[0,batch_array_t[nb]]
        drho_gt = traj_train_label_rho[0,batch_array_t[nb],0]
        
        traj_l2loss_vel = (accl[:,1:]-accl_gt).pow(2).mean()
        traj_l2loss_rho = (accl[:,0]-drho_gt).pow(2).mean()
        traj_l2loss = traj_l2loss_vel + traj_l2loss_rho
    
        # Statistics-based loss function
        kl = histtopdf(datatohist,accl_gt,accl[:,1:])
        kl_rho = histtopdf(datatohist_rho, drho_gt, accl[:,0])
        kl_loss = kl + kl_rho
        loss_stat = kl_loss

        loss = alpha_traj*traj_l2loss +  alpha_kl*loss_stat

        loss.backward()#retain_graph=True)
        optimizer.step()
        loss_traj.append(0)
        loss_traj.append(traj_l2loss.cpu().detach().numpy())
        loss_traj.append(kl_loss.cpu().detach().numpy())
    print('Epoch = {}, Total loss = {}'.format(i, loss.cpu().detach().numpy()))    
    print('Statistics loss = {}'.format(loss_stat.cpu().detach().numpy()))
    print('Traj loss for velocity= {}'.format(traj_l2loss_vel.cpu().detach().numpy()))
    scheduler.step()

Epoch = 0, Total loss = 2.9614447582325756
Statistics loss = 19.893892288654072
Traj loss for velocity= 0.3217294989795983


# Then train with Field-based loss

In [ ]:
for i in range(train_epochs):
    for nb in range(batch_number_f):
        optimizer.zero_grad()
        pred,accl,accl_gt,drho,drho_gt = model.cal_field_kl(pos_traj[0],vel_traj[:2],rho_traj[:2],batch_array_f[nb],w_field)
        
        l2loss = (pred-field_train_label[batch_array_f[nb]]).pow(2).mean()

        kl = histtopdf(datatohist,accl_gt,accl)
        kl_rho = histtopdf(datatohist_rho, drho_gt, drho)
        kl_loss = kl + kl_rho

        traj_l2loss_vel = (accl-accl_gt).pow(2).mean()
        traj_l2loss_rho = (drho-drho_gt).pow(2).mean()
        traj_l2loss = traj_l2loss_vel #+  traj_l2loss_rho  
        
        loss_stat = kl_loss

        loss = alpha_traj*traj_l2loss +  alpha_kl*loss_stat + alpha_field* l2loss

        loss.backward(retain_graph=True)
        optimizer.step()
        loss_traj.append(0)
        loss_traj.append(traj_l2loss.cpu().detach().numpy())
        loss_traj.append(kl_loss.cpu().detach().numpy())
        
        print('Epoch = {}, Total loss = {}'.format(i, loss.cpu().detach().numpy()))    
        print('Statistics loss = {}'.format(loss_stat.cpu().detach().numpy()))
        print('Traj loss for velocity= {}'.format(traj_l2loss_vel.cpu().detach().numpy()))
        scheduler.step()

Epoch = 0, Total loss = 1.2630326473619848
Statistics loss = 9.174406356758663
Traj loss for velocity= 0.3453333886934286
Epoch = 0, Total loss = 1.17862618142305
Statistics loss = 8.552518045892679
Traj loss for velocity= 0.32312925243265883
Epoch = 0, Total loss = 1.272332260713698
Statistics loss = 9.47429385599441
Traj loss for velocity= 0.3246603234322862
Epoch = 0, Total loss = 1.2934744685984028
Statistics loss = 9.48210454287661
Traj loss for velocity= 0.3450043408921831
Epoch = 0, Total loss = 1.2831193311212585
Statistics loss = 9.72877318849294
Traj loss for velocity= 0.31001738377683874
Epoch = 0, Total loss = 1.2423135755993049
Statistics loss = 9.357484193976475
Traj loss for velocity= 0.30632969385037145
Epoch = 0, Total loss = 1.3010495284785741
Statistics loss = 9.700304166108566
Traj loss for velocity= 0.3307763613797688
Epoch = 0, Total loss = 1.1719639099377956
Statistics loss = 8.574376599399542
Traj loss for velocity= 0.3142855928146275
Epoch = 0, Total loss = 1.3

In [ ]:
len(loss_traj)